In [1]:
base_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\ga_results\small_gap_30min\\'
benchmark_base_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\external_test_data\FJSSPinstances\\'
files = ['small_gap1.txt','small_gap2.txt','small_gap3.txt','small_gap4.txt','small_gap5.txt']

In [2]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [3]:
def translate(content, limit):
    line = dict()
    line['source'] = content[0]
    line['instance'] = int(content[1])
    line['best_objective'] = float(content[6])
    line['generations'] = int(content[4])
    line['restarts'] = int(content[5])
    #line['gap'] = float(columns[3])
    line['status'] = 2 if float(content[2]) < limit else 9 # NOTE: could still be optimal, just not known
    line['n_explored'] = int(float(content[3]))
    line['runtime'] = float(content[2])

    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
    line['limit'] = limit
    return line    

In [6]:
data = []
for file in files:
    content = open(base_path + file, 'r').readlines()
    for values in content:
        line = translate(values.split(';'), 1800)
        data.append(line)

In [7]:
import pandas as pd
df = pd.DataFrame.from_dict(data)

In [8]:
df

,source,instance,best_objective,generations,restarts,status,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
0,0_BehnkeGeiger,1,98.0,3234,17,9,3388290,1800.121810,20,10,6.08,50,5.0,0.304000,80.4,1800
1,0_BehnkeGeiger,2,104.0,3341,16,9,3578095,1800.454398,20,10,6.72,50,5.0,0.336000,83.6,1800
2,0_BehnkeGeiger,3,106.0,3245,17,9,3413590,1811.576850,20,10,6.24,50,5.0,0.312000,81.2,1800
3,0_BehnkeGeiger,4,110.0,3294,18,9,3338185,1838.008010,20,10,6.64,50,5.0,0.332000,83.2,1800
4,0_BehnkeGeiger,5,102.0,3383,17,9,3344610,1800.345015,20,10,6.16,50,5.0,0.308000,80.8,1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,6_Fattahi,14,564.0,2662,18,9,2866065,1800.048100,7,7,2.70,21,3.0,0.385714,29.1,1800
941,6_Fattahi,15,514.0,2464,16,2,2564195,1546.435491,7,7,2.60,21,3.0,0.371429,28.8,1800
942,6_Fattahi,16,634.0,1368,11,2,1051980,838.772738,7,8,2.60,24,3.0,0.371429,31.8,1800
943,6_Fattahi,17,881.0,2332,16,9,2355815,1800.251177,7,8,2.40,32,4.0,0.342857,41.6,1800


In [12]:
#average
instances = dict()
for result in data:
    if f'{result["source"]}{result["instance"]}' not in instances:
        instances[f'{result["source"]}{result["instance"]}'] = result
        instances[f'{result["source"]}{result["instance"]}']['min_objective'] = result["best_objective"]
        instances[f'{result["source"]}{result["instance"]}']['max_objective'] = result["best_objective"]
        
    else:
        instances[f'{result["source"]}{result["instance"]}']['best_objective'] = instances[f'{result["source"]}{result["instance"]}']['best_objective'] + result["best_objective"]
        instances[f'{result["source"]}{result["instance"]}']['generations'] = instances[f'{result["source"]}{result["instance"]}']['generations'] + result["generations"]
        instances[f'{result["source"]}{result["instance"]}']['restarts'] = instances[f'{result["source"]}{result["instance"]}']['restarts'] + result["restarts"]
        instances[f'{result["source"]}{result["instance"]}']['status'] = instances[f'{result["source"]}{result["instance"]}']['status'] + result["status"]
        instances[f'{result["source"]}{result["instance"]}']['n_explored'] = instances[f'{result["source"]}{result["instance"]}']['n_explored'] + result["n_explored"]
        instances[f'{result["source"]}{result["instance"]}']['runtime'] = instances[f'{result["source"]}{result["instance"]}']['runtime'] + result["runtime"]
        if result['best_objective'] < instances[f'{result["source"]}{result["instance"]}']['min_objective']:
            instances[f'{result["source"]}{result["instance"]}']['min_objective'] = result['best_objective']
        if result['best_objective'] > instances[f'{result["source"]}{result["instance"]}']['max_objective']:
            instances[f'{result["source"]}{result["instance"]}']['max_objective'] = result['best_objective']

for instance in instances:
    instances[instance]['best_objective'] = instances[instance]['best_objective'] / 5
    instances[instance]['generations'] = instances[instance]['generations'] / 5
    instances[instance]['restarts'] = instances[instance]['restarts'] / 5
    instances[instance]['status'] = instances[instance]['status'] / 5
    instances[instance]['n_explored'] = instances[instance]['n_explored'] / 5
    instances[instance]['runtime'] = instances[instance]['runtime'] / 5

df_avg = pd.DataFrame.from_dict(instances.values())


In [13]:
df_avg

,source,instance,best_objective,generations,restarts,status,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit,min_objective,max_objective
0,0_BehnkeGeiger,1,116.76,3512.88,18.60,10.44,3726453.2,2088.593795,20,10,6.08,50,5.0,0.304000,80.4,1800,99.0,180.8
1,0_BehnkeGeiger,2,119.52,3660.80,18.84,10.44,3757536.0,2125.067192,20,10,6.72,50,5.0,0.336000,83.6,1800,101.0,185.6
2,0_BehnkeGeiger,3,121.84,3510.52,18.88,10.44,3635607.0,2092.969711,20,10,6.24,50,5.0,0.312000,81.2,1800,104.0,189.2
3,0_BehnkeGeiger,4,124.24,3482.80,18.36,10.44,3744532.0,2089.784153,20,10,6.64,50,5.0,0.332000,83.2,1800,105.0,193.2
4,0_BehnkeGeiger,5,117.48,3497.28,18.88,10.44,3667590.2,2092.008650,20,10,6.16,50,5.0,0.308000,80.8,1800,98.0,182.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,6_Fattahi,14,648.64,2907.32,18.96,6.52,3056583.4,1584.509157,7,7,2.70,21,3.0,0.385714,29.1,1800,554.0,1007.2
185,6_Fattahi,15,596.24,1645.16,9.88,2.32,1113594.4,578.827883,7,7,2.60,21,3.0,0.371429,28.8,1800,514.0,925.2
186,6_Fattahi,16,735.44,1242.60,8.04,2.32,537414.0,353.892933,7,8,2.60,24,3.0,0.371429,31.8,1800,634.0,1141.2
187,6_Fattahi,17,1027.28,2598.92,16.76,8.48,2521831.2,1631.012747,7,8,2.40,32,4.0,0.342857,41.6,1800,879.0,1593.4


In [15]:
df_avg[df_avg["status"] < 9]

,source,instance,best_objective,generations,restarts,status,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit,min_objective,max_objective
15,1_Brandimarte,1,46.40,956.24,6.88,2.32,319796.6,334.864255,6,10,2.00,55,5.5,0.333333,66.00,1800,40.0,72.0
17,1_Brandimarte,3,236.64,924.04,4.64,2.32,96600.6,174.299909,8,15,3.00,150,10.0,0.375000,180.00,1800,204.0,367.2
21,2a_Hurink_sdata,4,772.56,417.76,2.52,2.32,35287.2,12.477716,5,10,1.00,50,5.0,0.200000,55.00,1800,666.0,1198.8
22,2a_Hurink_sdata,5,760.00,2126.40,13.16,2.60,1722542.0,1589.696025,5,10,1.00,50,5.0,0.200000,55.00,1800,655.0,1180.0
25,2a_Hurink_sdata,8,687.88,20.00,0.00,2.32,405.8,0.164336,5,10,1.00,50,5.0,0.200000,55.00,1800,593.0,1067.4
38,2a_Hurink_sdata,49,8164.08,672.64,3.92,2.32,59758.6,37.201918,5,11,1.00,55,5.0,0.200000,60.00,1800,7038.0,12668.4
39,2a_Hurink_sdata,50,8409.12,1695.56,12.32,6.52,1178070.2,1380.274367,4,13,1.00,52,4.0,0.250000,56.00,1800,7166.0,13054.6
54,2b_Hurink_edata,1,63.80,126.36,0.56,2.32,3291.8,0.643616,6,6,1.15,36,6.0,0.191667,42.90,1800,55.0,99.0
56,2b_Hurink_edata,4,706.44,991.76,6.80,2.32,341477.8,338.727093,5,10,1.15,50,5.0,0.230000,55.75,1800,609.0,1096.2
57,2b_Hurink_edata,5,759.80,1140.72,8.04,2.32,397824.4,436.150586,5,10,1.15,50,5.0,0.230000,55.75,1800,655.0,1179.0
